# 📊 Uncertainty Quantification

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Information</b><br>
<b>Level:</b> Intermediate/Advanced<br>
<b>Time:</b> 20 minutes<br>
<b>Dataset:</b> California Housing (sklearn)<br>
<b>Prerequisite:</b> 01_tests_introduction.ipynb
</div>

## 🎯 Objectives
- ✅ Understand why uncertainty matters
- ✅ Learn about CRQR (Conformalized Quantile Regression)
- ✅ Generate confidence intervals
- ✅ Analyze probability calibration
- ✅ Make uncertainty-based decisions

## 📚 Why Does Uncertainty Matter?

### Critical Contexts

#### 🏥 Medicine
- **Problem**: Cancer diagnosis
- **Solution**: "90% confidence it's malignant" vs "not sure"
- **Impact**: Save lives with informed decisions

#### 💰 Finance
- **Problem**: Credit approval
- **Solution**: Quantify default risk
- **Impact**: More informed risk decisions

#### 🚗 Autonomous Vehicles
- **Problem**: Pedestrian detection
- **Solution**: Know when not confident
- **Impact**: Increased safety

### The Problem with Traditional Models
```python
# Traditional prediction
prediction = model.predict(X)  # [0.92]
# But how confident are we?
# 92% ± 1%? or 92% ± 20%?
```

## 1️⃣ Setup - Regression Problem

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from deepbridge import DBDataset, Experiment

# Configure visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load data (house prices)
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['target'] = housing.target  # Price in $100k

# Use subset for speed
df = df.sample(n=5000, random_state=42)

print(f"📊 Dataset: {df.shape}")
print(f"🏠 Target: House prices (in $100k)")
print(f"\n📈 Target statistics:")
print(df['target'].describe())

## 2️⃣ Train Regression Model

In [ ]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train RandomForest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"✅ Model trained!")
print(f"📊 MAE: ${mae*100:.2f}k")
print(f"📊 R²: {r2:.3f}")

## 3️⃣ Create Experiment

In [ ]:
dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=42,
    dataset_name='California Housing Prices'
)

exp = Experiment(
    dataset=dataset,
    experiment_type='regression',
    random_state=42
)

print("✅ Experiment created!")

## 4️⃣ Run Uncertainty Test

<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; border-left: 5px solid #ff9800;">
<b>ℹ️ CRQR:</b> Conformalized Quantile Regression is an advanced technique for generating calibrated prediction intervals.
</div>

In [ ]:
print("🧪 Running uncertainty quantification test...\n")

uncertainty_result = exp.run_test('uncertainty', config_name='quick')

print("\n✅ Uncertainty test completed!")

## 5️⃣ Confidence Intervals

Now we have confidence intervals for each prediction!

In [ ]:
# Extract intervals (if available)
if 'prediction_intervals' in uncertainty_result:
    intervals = uncertainty_result['prediction_intervals']
    
    # Example: first 5 predictions
    print("\n📊 CONFIDENCE INTERVALS (first 5 predictions):\n" + "="*70)
    
    for i in range(min(5, len(intervals))):
        lower = intervals[i]['lower_bound']
        upper = intervals[i]['upper_bound']
        prediction = intervals[i]['prediction']
        actual = y_test.iloc[i] if i < len(y_test) else None
        
        print(f"\n🏠 House {i+1}:")
        print(f"   Prediction: ${prediction*100:.2f}k")
        print(f"   Interval: [${lower*100:.2f}k, ${upper*100:.2f}k]")
        print(f"   Width: ${(upper-lower)*100:.2f}k")
        if actual is not None:
            print(f"   Actual Value: ${actual*100:.2f}k")
            contains = lower <= actual <= upper
            print(f"   Contains actual? {'✅' if contains else '❌'}")
else:
    print("\n💡 Prediction intervals:\n")
    print("For each prediction, we have an interval [lower, upper]:")
    print("\nExample:")
    print("  Prediction: $250k")
    print("  95% Interval: [$200k, $300k]")
    print("  Interpretation: 95% confidence that the actual value is within this interval")

## 6️⃣ Coverage Analysis

**Coverage** = % of actual values that fall within predicted intervals

In [ ]:
if 'coverage' in uncertainty_result:
    coverage = uncertainty_result['coverage']
    target_coverage = uncertainty_result.get('target_coverage', 0.95)
    
    print(f"\n📊 COVERAGE ANALYSIS\n" + "="*60)
    print(f"\n🎯 Target Coverage: {target_coverage*100:.0f}%")
    print(f"✅ Achieved Coverage: {coverage*100:.1f}%")
    
    # Evaluate quality
    if abs(coverage - target_coverage) < 0.05:
        print("\n🟢 EXCELLENT - Coverage is calibrated!")
    elif abs(coverage - target_coverage) < 0.10:
        print("\n🟡 GOOD - Acceptable coverage")
    else:
        print("\n🔴 WARNING - Coverage is miscalibrated")
        
    print(f"\n💡 Interpretation:")
    print(f"   {coverage*100:.1f}% of predictions contain the actual value in the interval")
else:
    print("\n📊 Expected coverage: ~95%")
    print("   Means that 95% of actual values should fall within the intervals")

## 7️⃣ Visualize Confidence Intervals

In [ ]:
# Create interval visualization
n_samples = 30  # Show 30 samples

# Simulate intervals if not available
if 'prediction_intervals' not in uncertainty_result:
    # Simulate for demonstration
    predictions = y_pred[:n_samples]
    std_pred = np.std(y_test[:n_samples] - predictions)
    lower = predictions - 1.96 * std_pred
    upper = predictions + 1.96 * std_pred
    actual = y_test.iloc[:n_samples].values
else:
    intervals = uncertainty_result['prediction_intervals'][:n_samples]
    predictions = np.array([i['prediction'] for i in intervals])
    lower = np.array([i['lower_bound'] for i in intervals])
    upper = np.array([i['upper_bound'] for i in intervals])
    actual = y_test.iloc[:n_samples].values

# Plot
plt.figure(figsize=(14, 6))

x = np.arange(n_samples)

# Confidence intervals
plt.fill_between(x, lower, upper, alpha=0.3, color='skyblue', 
                 label='95% Confidence Interval')

# Predictions
plt.plot(x, predictions, 'o-', color='blue', label='Prediction', 
         markersize=6, linewidth=2)

# Actual values
plt.scatter(x, actual, color='red', marker='x', s=100, 
            label='Actual Value', zorder=5, linewidth=2)

plt.xlabel('Sample', fontsize=12)
plt.ylabel('Price ($100k)', fontsize=12)
plt.title('Confidence Intervals vs Actual Values', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Count how many actual values fall within intervals
contains = np.sum((actual >= lower) & (actual <= upper))
coverage_calc = contains / n_samples

print(f"\n📊 In the {n_samples} samples shown:")
print(f"   ✅ {contains}/{n_samples} actual values within interval")
print(f"   📊 Coverage: {coverage_calc*100:.1f}%")

## 8️⃣ Probability Calibration

For classification problems, we can also evaluate calibration:

In [ ]:
print("\n📊 PROBABILITY CALIBRATION\n" + "="*60)

print("\n💡 What is calibration?")
print("   If the model says '70% chance', it should be right ~70% of the time")
print("\n✅ Calibrated model:")
print("   - Prob = 0.9 → 90% accuracy")
print("   - Prob = 0.7 → 70% accuracy")
print("   - Prob = 0.5 → 50% accuracy")

print("\n❌ Miscalibrated model:")
print("   - Prob = 0.9 → 60% accuracy (overconfident)")
print("   - Prob = 0.5 → 80% accuracy (underconfident)")

if 'calibration_score' in uncertainty_result:
    cal_score = uncertainty_result['calibration_score']
    print(f"\n📊 Calibration Score: {cal_score:.3f}")
    
    if cal_score > 0.9:
        print("🟢 Excellent calibration!")
    elif cal_score > 0.7:
        print("🟡 Acceptable calibration")
    else:
        print("🔴 Calibration needs improvement")

## 9️⃣ Uncertainty-Based Decisions

How to use uncertainty in practice?

In [ ]:
print("\n💼 PRACTICAL USE EXAMPLES\n" + "="*60)

print("\n🏥 MEDICINE - Cancer Diagnosis")
print("   Prediction: 85% chance of cancer")
print("   Interval: [60%, 95%]")
print("   Action: ✅ Perform biopsy (high uncertainty in critical decision)")

print("\n💰 FINANCE - Credit Approval")
print("   Prediction: 40% chance of default")
print("   Interval: [35%, 45%]")
print("   Action: ✅ Approve with adjusted rate (low uncertainty)")

print("\n🚗 AUTONOMOUS VEHICLES - Pedestrian Detection")
print("   Prediction: 70% chance it's a pedestrian")
print("   Interval: [30%, 90%]")
print("   Action: ⚠️  Brake preventively (high uncertainty = caution)")

print("\n🏠 OUR EXAMPLE - House Price")
sample_idx = 0
if 'prediction_intervals' in uncertainty_result and len(uncertainty_result['prediction_intervals']) > 0:
    interval = uncertainty_result['prediction_intervals'][sample_idx]
    pred = interval['prediction']
    low = interval['lower_bound']
    up = interval['upper_bound']
    width = up - low
else:
    pred = y_pred[sample_idx]
    std = np.std(y_test - y_pred[:len(y_test)])
    low = pred - 1.96 * std
    up = pred + 1.96 * std
    width = up - low

print(f"   Prediction: ${pred*100:.2f}k")
print(f"   Interval: [${low*100:.2f}k, ${up*100:.2f}k]")
print(f"   Width: ${width*100:.2f}k")

if width < 1.0:  # $100k
    print(f"   Action: ✅ High confidence - can use prediction directly")
else:
    print(f"   Action: ⚠️  High uncertainty - consider range of values")

## 🔟 Comparison: With vs Without Uncertainty

In [ ]:
print("\n📊 WITHOUT UNCERTAINTY QUANTIFICATION:\n" + "="*60)
print("❌ Prediction: $250k")
print("❌ Problem: We don't know how reliable it is!")
print("❌ Risk: Decisions without confidence context")

print("\n📊 WITH UNCERTAINTY QUANTIFICATION:\n" + "="*60)
print("✅ Prediction: $250k")
print("✅ 95% Interval: [$200k, $300k]")
print("✅ Benefit: We know the margin of error!")
print("✅ Decisions: More informed and safe")

print("\n💡 IMPACT:")
print("   🏥 Medicine: Avoid wrong diagnoses")
print("   💰 Finance: Manage risk appropriately")
print("   🚗 Autonomy: Prioritize safety in uncertain situations")
print("   🏠 Real Estate: Negotiate with realistic value range")

## 🎉 Conclusion

### What you learned:
- ✅ **Importance of Uncertainty** - Essential for critical decisions
- ✅ **CRQR** - State-of-the-art technique for calibrated intervals
- ✅ **Confidence Intervals** - Quantify margin of error
- ✅ **Coverage** - Validate interval calibration
- ✅ **Practical Applications** - How to use in different domains

### Uncertainty Checklist for Production:
- [ ] ✅ Coverage close to target (e.g., 95%)
- [ ] ✅ Calibrated intervals
- [ ] ✅ Interval width acceptable for business
- [ ] ✅ Uncertainty-based decision rules defined
- [ ] ✅ Calibration monitoring in production

### When to Use Uncertainty Quantification:
- 🏥 **Always** in medical applications
- 💰 **Always** in financial decisions
- 🚗 **Always** in safety systems
- 📊 **Recommended** in any critical application

### Next Steps:
- 📘 `04_resilience_drift.ipynb` - Detect data changes
- 📘 `../04_fairness/` - Fairness and bias analysis

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4caf50;">
<b>💡 Remember:</b> "Being wrong with confidence is worse than being uncertain" - always quantify uncertainty!
</div>